# Etapa 2 — Limpieza, Enriquecimiento y EDA  
**Proyecto Integrado V — Ingeniería de Software y Datos (IU Digital de Antioquia)**

**Dataset:** `datos/estilo de vida_salud_kaggle.csv`  
**Variables clave seleccionadas (Etapa 1):**  
- `age`  
- `bmi`  
- `exercise_days_per_week`  
- `sleep_hours`  
- `sugar_intake_g`

---

## Objetivo de la etapa  
1. Ejecutar limpieza, normalización y enriquecimiento del dataset con Python.  
2. Agregar columnas de fecha, año, mes y día.  
3. Realizar análisis descriptivo (EDA) de las 5 variables clave mediante estadísticas y visualizaciones.  
4. Guardar dataset enriquecido y evidencias gráficas en el repositorio.  

> **Nota metodológica:** si el dataset no contiene fecha, se genera una columna de fecha aleatoria (2022–2024) para análisis temporal exploratorio.


## 1. Configuración del entorno  
En esta sección se importan librerías, se definen rutas del repositorio y variables clave.  
Cada paso queda registrado en el notebook con comentarios y salidas de evidencia (prints).

In [ ]:
import os
import re
import unicodedata
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# =========================
# Configuración base (AJUSTADA A TU REPO)
# =========================
# Entrada:
RUTA_DATASET_ORIGINAL = Path("datos/estilo de vida_salud_kaggle.csv")

# Salidas:
RUTA_DATASET_ENRIQ = Path("datos/dataset_enriquecido.csv")
RUTA_GRAFICOS = Path("docs/graficos")

# Variables clave seleccionadas (Etapa 1)
VARS_CLAVE = [
    "age",
    "bmi",
    "exercise_days_per_week",
    "sleep_hours",
    "sugar_intake_g"
]

# Opcional: manejo sencillo de outliers
HANDLE_OUTLIERS = False  # cambia a True si quieres winsorizar por IQR

# Crear carpetas destino si no existen
RUTA_GRAFICOS.mkdir(parents=True, exist_ok=True)
RUTA_DATASET_ENRIQ.parent.mkdir(parents=True, exist_ok=True)


## 2. Funciones auxiliares  
Se definen funciones para:  
- Normalizar nombres de columnas.  
- Leer CSV con diferentes codificaciones.  
- Mapear aliases de columnas de Kaggle a nombres estándar.  
- Reportar calidad de datos.  
- Manejar nulos y (opcional) outliers.  
- Generar/derivar fechas.  
- Crear y guardar gráficos.  


In [ ]:
# =========================
# Helpers de limpieza
# =========================
def strip_accents(text: str) -> str:
    """Elimina acentos/diacríticos."""
    if not isinstance(text, str):
        return text
    text = unicodedata.normalize("NFD", text)
    text = "".join(ch for ch in text if unicodedata.category(ch) != "Mn")
    return text

def normalize_column_name(col: str) -> str:
    """Normaliza nombre de columna: minúsculas, sin acentos, sin espacios, solo _."""
    col = strip_accents(col)
    col = col.lower().strip()
    col = re.sub(r"\s+", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col)
    return col

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Aplica normalización a todas las columnas."""
    old_cols = df.columns.tolist()
    df.columns = [normalize_column_name(c) for c in df.columns]
    print("✅ Columnas normalizadas.")
    print("Antes:", old_cols)
    print("Después:", df.columns.tolist())
    return df

def try_read_csv(path: Path) -> pd.DataFrame:
    """Lee CSV intentando codificaciones comunes."""
    if not path.exists():
        raise FileNotFoundError(f"No se encontró el archivo en: {path.resolve()}")
    encodings = ["utf-8", "latin-1", "cp1252"]
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc)
            print(f"✅ Dataset leído con encoding {enc}. Shape: {df.shape}")
            return df
        except Exception as e:
            last_err = e
    raise last_err

def map_aliases(df: pd.DataFrame) -> pd.DataFrame:
    """
    Mapea posibles variantes de nombres a estándar.
    Útil si Kaggle trae columnas ligeramente distintas.
    """
    aliases = {
        "age": ["age", "edad"],
        "bmi": ["bmi", "imc", "indice_de_masa_corporal"],
        "exercise_days_per_week": [
            "exercise_days_per_week", "exercise_days", "exercise_frequency",
            "dias_ejercicio_semana", "exercise_per_week"
        ],
        "sleep_hours": ["sleep_hours", "hours_sleep", "horas_sueno", "sleep_per_day"],
        "sugar_intake_g": [
            "sugar_intake_g", "sugar_intake", "sugar_g", "ingesta_azucar",
            "sugar_consumption_g"
        ],
    }

    inverse = {}
    for std, al_list in aliases.items():
        for al in al_list:
            inverse[al] = std

    rename_dict = {}
    for c in df.columns:
        if c in inverse and inverse[c] != c:
            rename_dict[c] = inverse[c]

    if rename_dict:
        df = df.rename(columns=rename_dict)
        print("✅ Aliases detectados y renombrados:", rename_dict)
    else:
        print("ℹ️ No se detectaron aliases a renombrar.")
    return df

def report_missing(df: pd.DataFrame):
    miss = df.isna().mean().sort_values(ascending=False)
    print("\n📌 Porcentaje de nulos por columna (top 10):")
    print((miss.head(10) * 100).round(2))

def coerce_numeric(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """Convierte columnas a numéricas forzando errores a NaN."""
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def handle_missing_key_vars(df: pd.DataFrame, cols: list, drop_thresh=0.05):
    """
    Si nulos <= drop_thresh -> elimina filas.
    Si nulos > drop_thresh -> imputa con mediana.
    """
    for c in cols:
        if c not in df.columns:
            continue
        ratio = df[c].isna().mean()
        if ratio <= drop_thresh:
            before = df.shape[0]
            df = df.dropna(subset=[c])
            after = df.shape[0]
            print(f"✅ {c}: nulos {ratio:.2%} -> se eliminaron {before-after} filas.")
        else:
            med = df[c].median()
            df[c] = df[c].fillna(med)
            print(f"✅ {c}: nulos {ratio:.2%} -> imputación mediana={med:.3f}.")
    return df

def winsorize_iqr(df: pd.DataFrame, cols: list, k=1.5):
    """Recorta outliers con límites IQR."""
    for c in cols:
        if c not in df.columns:
            continue
        q1 = df[c].quantile(0.25)
        q3 = df[c].quantile(0.75)
        iqr = q3 - q1
        low = q1 - k * iqr
        high = q3 + k * iqr
        df[c] = df[c].clip(lower=low, upper=high)
        print(f"✅ Outliers tratados en {c} con IQR.")
    return df

# =========================
# Enriquecimiento temporal
# =========================
def detect_date_column(df: pd.DataFrame):
    possibles = ["date", "fecha", "fecha_registro", "recorded_date", "timestamp"]
    for c in possibles:
        if c in df.columns:
            return c
    return None

def add_random_date(df: pd.DataFrame, start="2022-01-01", end="2024-12-31", seed=42):
    """Genera columna fecha aleatoria dentro del rango."""
    np.random.seed(seed)
    start_dt = pd.to_datetime(start)
    end_dt = pd.to_datetime(end)
    days_range = (end_dt - start_dt).days

    random_days = np.random.randint(0, days_range + 1, size=len(df))
    df["fecha"] = start_dt + pd.to_timedelta(random_days, unit="D")
    print("✅ Columna 'fecha' generada aleatoriamente.")
    return df

def derive_date_parts(df: pd.DataFrame, date_col="fecha"):
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df["anio"] = df[date_col].dt.year
    df["mes"] = df[date_col].dt.month
    df["dia"] = df[date_col].dt.day
    print("✅ Columnas anio, mes, dia derivadas.")
    return df

# =========================
# EDA + gráficos
# =========================
def save_fig(path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()

def plot_histograms(df: pd.DataFrame, cols: list):
    for c in cols:
        if c not in df.columns:
            continue
        plt.figure()
        df[c].hist(bins=30)
        plt.title(f"Histograma de {c}")
        plt.xlabel(c)
        plt.ylabel("Frecuencia")
        save_fig(RUTA_GRAFICOS / f"hist_{c}.png")
        print(f"📈 Histograma guardado: hist_{c}.png")

def plot_bar_exercise(df: pd.DataFrame):
    c = "exercise_days_per_week"
    if c not in df.columns:
        return
    plt.figure()
    df[c].value_counts().sort_index().plot(kind="bar")
    plt.title("Frecuencia de días de ejercicio por semana")
    plt.xlabel("Días por semana")
    plt.ylabel("Número de personas")
    save_fig(RUTA_GRAFICOS / "bar_exercise_days_per_week.png")
    print("📊 Barras guardadas: bar_exercise_days_per_week.png")

def plot_corr_heatmap(df: pd.DataFrame, cols: list):
    data = df[cols].dropna()
    corr = data.corr()

    plt.figure()
    plt.imshow(corr, interpolation="nearest")
    plt.colorbar()
    plt.xticks(range(len(cols)), cols, rotation=45, ha="right")
    plt.yticks(range(len(cols)), cols)
    plt.title("Matriz de correlación (variables clave)")
    for i in range(len(cols)):
        for j in range(len(cols)):
            plt.text(j, i, f"{corr.iloc[i, j]:.2f}",
                     ha="center", va="center")
    save_fig(RUTA_GRAFICOS / "corr_heatmap_vars_clave.png")
    print("🔥 Heatmap guardado: corr_heatmap_vars_clave.png")

def plot_time_series(df: pd.DataFrame, cols: list):
    if "anio" not in df.columns:
        return
    agg = df.groupby("anio")[cols].mean(numeric_only=True)

    plt.figure()
    for c in cols:
        plt.plot(agg.index, agg[c], marker="o", label=c)
    plt.title("Promedio anual de variables clave")
    plt.xlabel("Año")
    plt.ylabel("Promedio")
    plt.legend()
    save_fig(RUTA_GRAFICOS / "ts_promedios_anuales.png")
    print("⏱️ Serie temporal guardada: ts_promedios_anuales.png")

def generar_interpretaciones(df: pd.DataFrame, cols: list):
    """Genera interpretaciones breves en Markdown para cada gráfico."""
    lines = []
    lines.append("# Interpretaciones breves de gráficos (Etapa 2)\n")

    for c in ["age", "bmi", "sleep_hours", "sugar_intake_g"]:
        if c not in df.columns:
            continue
        s = df[c].dropna()
        lines.append(f"## hist_{c}.png")
        lines.append(
            f"- Media: {s.mean():.2f}, mediana: {s.median():.2f}, desviación: {s.std():.2f}.\n"
            f"- La distribución se concentra alrededor de {s.median():.2f} "
            f"con un rango aproximado entre {s.min():.2f} y {s.max():.2f}.\n"
        )

    if "exercise_days_per_week" in df.columns:
        vc = df["exercise_days_per_week"].value_counts().sort_index()
        moda = vc.idxmax()
        lines.append("## bar_exercise_days_per_week.png")
        lines.append(
            f"- El valor más frecuente es {moda} días/semana.\n"
            f"- Se observa concentración de participantes alrededor de ese nivel de actividad.\n"
        )

    corr = df[cols].corr()
    max_pair = corr.where(~np.eye(len(cols), dtype=bool)).stack().idxmax()
    max_val = corr.loc[max_pair]
    lines.append("## corr_heatmap_vars_clave.png")
    lines.append(
        f"- La relación más fuerte se observa entre **{max_pair[0]}** y **{max_pair[1]}** "
        f"con correlación ≈ {max_val:.2f}.\n"
        "- Esto sugiere asociación entre hábitos/condiciones sin implicar causalidad.\n"
    )

    if "anio" in df.columns:
        lines.append("## ts_promedios_anuales.png")
        lines.append(
            "- El gráfico temporal compara promedios por año (simulado).\n"
            "- Permite observar tendencias generales de hábitos/condiciones en el periodo.\n"
        )

    out_path = RUTA_GRAFICOS / "interpretaciones.md"
    out_path.write_text("\n".join(lines), encoding="utf-8")
    print(f"📝 Interpretaciones guardadas en: {out_path}")


## 3. Carga y limpieza de datos  
En esta sección se realiza:

1. Carga del CSV original.  
2. Normalización de nombres de columnas.  
3. Eliminación de duplicados.  
4. Conversión de tipos a numéricos en variables clave.  
5. Tratamiento de nulos con reglas explícitas.  
6. (Opcional) tratamiento de outliers.  

Cada decisión queda evidenciada con impresiones en consola.  


In [ ]:
# 1) Cargar dataset original
df = try_read_csv(RUTA_DATASET_ORIGINAL)

# 2) Normalizar columnas
df = normalize_columns(df)

# 3) Mapear aliases a nombres estándar
df = map_aliases(df)

# 4) Reporte inicial de calidad
print("\n📌 Shape inicial:", df.shape)
report_missing(df)

# 5) Eliminar duplicados
dup = df.duplicated().sum()
if dup > 0:
    df = df.drop_duplicates()
    print(f"✅ Duplicados eliminados: {dup}")
else:
    print("ℹ️ No se encontraron duplicados.")

# 6) Conversión de tipos a numéricos en variables clave
df = coerce_numeric(df, VARS_CLAVE)

# 7) Manejo de nulos en variables clave (drop si pocos, imputar si muchos)
df = handle_missing_key_vars(df, VARS_CLAVE, drop_thresh=0.05)

# 8) Manejo opcional de outliers
if HANDLE_OUTLIERS:
    df = winsorize_iqr(df, VARS_CLAVE)

# Reporte posterior a limpieza
print("\n📌 Shape después de limpieza:", df.shape)
report_missing(df)

df.head()


## 4. Enriquecimiento temporal  
Se verifica si existe columna de fecha.  
- Si existe: se derivan `anio`, `mes`, `dia`.  
- Si no existe: se genera una fecha aleatoria 2022–2024 y luego se derivan las columnas.  


In [ ]:
date_col = detect_date_column(df)
if date_col:
    print(f"✅ Columna fecha detectada: {date_col}")
    df = derive_date_parts(df, date_col)
else:
    df = add_random_date(df, start="2022-01-01", end="2024-12-31", seed=42)
    df = derive_date_parts(df, "fecha")

df[["fecha", "anio", "mes", "dia"]].head()


## 5. Exportación del dataset enriquecido  
El dataset resultante se guarda como:  
`datos/dataset_enriquecido.csv`  


In [ ]:
df.to_csv(RUTA_DATASET_ENRIQ, index=False)
print(f"✅ Dataset enriquecido guardado en: {RUTA_DATASET_ENRIQ}")

RUTA_DATASET_ENRIQ


## 6. Estadísticas descriptivas (describe)  
Se calculan estadísticas generales para las 5 variables clave y se exporta un CSV de evidencia.  


In [ ]:
describe_df = df[VARS_CLAVE].describe()
describe_df


In [ ]:
describe_path = RUTA_GRAFICOS / "describe_selected.csv"
describe_df.to_csv(describe_path)
print(f"🧾 Describe guardado en: {describe_path}")


## 7. Visualizaciones EDA  
Se generan y guardan en `docs/graficos/`:

- Histogramas: `age`, `bmi`, `sleep_hours`, `sugar_intake_g`.  
- Barras: `exercise_days_per_week`.  
- Correlación: heatmap con las 5 variables clave.  
- Temporal: promedios por año.  

> Después de ejecutar, revisa la carpeta `docs/graficos/` y copia/pega las interpretaciones en tu documento APA.  


In [ ]:
plot_histograms(df, ["age", "bmi", "sleep_hours", "sugar_intake_g"])
plot_bar_exercise(df)
plot_corr_heatmap(df, VARS_CLAVE)
plot_time_series(df, VARS_CLAVE)

print("✅ Gráficos generados. Revisa docs/graficos/")


## 8. Interpretaciones breves  
Para cumplir la rúbrica, se genera automáticamente un archivo con interpretaciones cortas:  
`docs/graficos/interpretaciones.md`  

Puedes usar ese texto “debajo de cada gráfico” en el notebook o en el documento APA.  


In [ ]:
generar_interpretaciones(df, VARS_CLAVE)


---

## 9. Conclusiones parciales de Etapa 2  
(Escribe aquí 1–2 párrafos cortos sobre qué encontraste en las 5 variables clave y qué tendencias/relaciones te parecen importantes.)

- ✅ Limpieza y normalización completadas.  
- ✅ Columnas fecha, anio, mes y dia agregadas.  
- ✅ Dataset enriquecido exportado.  
- ✅ EDA y gráficos generados con evidencia.  
- ✅ Interpretaciones listas para el APA.  
